## Intoduction

All the ouput files from the condor jobs are kept in a local directory


Paths to these files are stored in `fileLists/*.fls`

`Util.h` header is required for the isGoodTurnOn and getArea function


# Things To Note

### The Important parameters for this specific workflow are :
 1. EffMin and EffMax for finding the isGoodTurnOn criteria

### Always check what are the conditions used for exporting the results to the rootfile / summary file 
```
By default , The summary file found outside all the rate folders will have top five options that pass the isGoodTurnOn criteria .. but for the top 5 options for each unique pT  at a fixed rate this condition is not imposed. So please edit the exporting part according to what you would like to have
```

## Description of the files in the output
```
folder : $prefixBase 
            --> Contains a summaryForFixedRateScans.txt 
              --> Has the top 5 options for each fixed rates [ ranked in 'metric' which defaults to area ]
              --> For each fixed rate what are the unique pT thresholds that can be obtained 
              --> Details of the baseline identified for the fixed rate
            --> Contains directories for various fixed rates being searched
              --> Contains allOptionsScan.txt : details of all the options considered [ area / threshold ]
              --> Contains allOptions.root
                 * Contains the turnon at fixed rate for all options 
                 * Contains the rate vs pT histogram for all options
                 * can optionally uncomment 2 lines in the export part to add all the turnons for each option
                   ++ Note this will make the size of the allOptions.root file > 150 MB per fixed rate
              --> Contains selectedOptions.txt : details of all the options that pass isGoodTurnOn
              --> Contains selectedOptions.root
                 * Contains the top 5 options that pass isGoodTurnOn 
                      + Rates / turnOn at Fixed Rate / turnOns at all eTs for these options
                 * Contains the top 3 options for each unique eT identified for this fixed rate
                      + Rates / turnOn at Fixed Rate / turnOns at all eTs for these options
```                 

### Reading the Files

In [ ]:
import numpy as np
import os
import ROOT as rt

In [ ]:
rt.gInterpreter.ProcessLine('#include "Util.h"')

# Define the dataset Used

In [ ]:
RateStudyNtuple=' Single Neutrino Gun, 122X, re emulated with the latest 12_3_0_pre6, caloparams_2022_0_v2, Final recaliberated eT LUT, 1M evts'
EffStudyNtuple='DYZToll , 122X, re emulated with the latest 12_3_0_pre6, caloparams_2022_0_v2,  Final recaliberated eT LUT,1M evts'

DatsetInfoStr='Rates studied using \n\t:\t '+RateStudyNtuple+'\n'
DatsetInfoStr+='Efficiency studied using \n\t:\t '+EffStudyNtuple+'\n'

Run2RateFile='data/HistgramFile_Rate_Run3MC_2022v0_2_recalibV1_withPFA1.root'
# Run2TurnOnFile='data/run3MC_122X_v1p0_2022_v0_2_turnon.root'
Run2TurnOnFile='data/TagAndProbe_MC_122Xturnon.root'
OtherInfo ='\tBaseline Turnons : '+str(os.path.abspath(Run2TurnOnFile))
OtherInfo+='\n\tBaseline Rates : '+str(os.path.abspath(Run2TurnOnFile))

acceptanceTol=0.85

OtherInfo ='\tBaseline Turnons : '+str(os.path.abspath(Run2TurnOnFile))
OtherInfo+='\n\tBaseline Rates : '+str(os.path.abspath(Run2TurnOnFile))

effMin=0.85
effMax=0.95


condorFileList='data/gridC_v2.laptop.fls'
prefixBase="results/isoDevV4sigmoid/DoubleEG/FixedRateScans"

condorFileList='data/gridV4_122X_withPFA1p.fls'
prefixBase="results/isoDevV4_withPFA1pGridA/DoubleEG/FixedRateScans_v2"


In [ ]:
def getOptionDescFromRateProgressionName(rateProgressionName):
    items=rateProgressionName.split('_')
    desc={}
    desc['option']=int(items[3].replace('option',''))
    desc['option_str']='_'.join(items[3:7])
    desc['optionParams']=(float(items[4].replace('p','.')),
                          float(items[5].replace('p','.')),
                          float(items[6].replace('p','.')))
    desc['EtThreshold'] = float(items[8].replace('p','.'))
    

    return desc
def getThresholdRate(rate):
    return  float( int((rate+0.1)*10)/10.0 )

## Definie the Main data storege 

#### Load the Et and Rate metrics

In [ ]:
def updateDataStore(data,RateForTurnons,AcceptanceForTurnons,TurnOnProgressionDir,RateProgressionDir):
    xAxisForNames = AcceptanceForTurnons.GetXaxis();
    nBins = AcceptanceForTurnons.GetNbinsX();
    AcceptanceMap={}
    for i in range(nBins):
        option_histName=xAxisForNames.GetBinLabel(i)
        if(option_histName==''):
            continue
        AcceptanceMap[option_histName]=AcceptanceForTurnons.GetBinContent(i)
        
    TurnOnMap={}    
    for ky in  TurnOnProgressionDir.GetListOfKeys():
        hName=ky.GetName()
        TurnOnMap[hName]=TurnOnProgressionDir.Get(hName)
    
    RateMap={}    
    for ky in  RateProgressionDir.GetListOfKeys():
        hName=ky.GetName()
#         print(hName)
#         print(hName.replace('rate_Progression_double','').split('_'))
        opt=hName.replace('rate_Progression_double','').split('_')[0]
        if opt=='':
            opt=int(hName.replace('rate_Progression_double','').split('_')[1])
        opt=int(opt)
        RateMap[opt]=RateProgressionDir.Get(hName)
        
    xAxisForNames = RateForTurnons.GetXaxis();
    nBins = RateForTurnons.GetNbinsX();
    for i in range(nBins):
        option_histName=xAxisForNames.GetBinLabel(i)
        if(option_histName==''):
            continue

        rate=RateForTurnons.GetBinContent(i)
        optionDesc=getOptionDescFromRateProgressionName(option_histName)

        opt=optionDesc['option']
        eT=optionDesc['EtThreshold']
        rateProgression=RateMap[opt]
        
        if opt not in data:
            data[opt]={}
            data[opt]['params']=optionDesc['optionParams']
            data[opt]['option_str']=optionDesc['option_str']
            data[opt]['rateProgression']=rateProgression

        acceptance=AcceptanceMap[option_histName]
        turnOn=TurnOnMap[option_histName]
        area=9.0
        area=rt.getIntegral(turnOn,eT,eT-7.0,eT+7.0,"",rt.nullptr,False)
        isGoodTurnOn=True
        
        if 'FixedEtMetrics' not in data[opt]:
            data[opt]['FixedEtMetrics']={}
        if eT not in data[opt]['FixedEtMetrics']:
                data[opt]['FixedEtMetrics'][eT]={}
        data[opt]['FixedEtMetrics'][eT]['rate']=rate
        data[opt]['FixedEtMetrics'][eT]['acceptance']=acceptance
        data[opt]['FixedEtMetrics'][eT]['turnOn']=turnOn
        data[opt]['FixedEtMetrics'][eT]['area']=area
        data[opt]['FixedEtMetrics'][eT]['isGoodTurnOn']=isGoodTurnOn

        if 'FixedRateMetrics' not in data[opt]:
            data[opt]['FixedRateMetrics']={}
        rate_rounded=getThresholdRate(rate)
       
        if rate_rounded in data[opt]['FixedRateMetrics']: 
            if (data[opt]['FixedRateMetrics'][rate_rounded]['eT'] < eT):
                pass
        else :
            data[opt]['FixedRateMetrics'][rate_rounded]={}
            data[opt]['FixedRateMetrics'][rate_rounded]['eT']=eT
            data[opt]['FixedRateMetrics'][rate_rounded]['acceptance']=acceptance
            data[opt]['FixedRateMetrics'][rate_rounded]['_rate']=rate
            data[opt]['FixedRateMetrics'][rate_rounded]['turnOn']=turnOn
            data[opt]['FixedRateMetrics'][rate_rounded]['area']=area
            data[opt]['FixedRateMetrics'][rate_rounded]['isGoodTurnOn']=isGoodTurnOn

    return data

In [ ]:
data={}
TFILE_STORE={}
count=0
for fName in fileNames:
    count+=1
    print(count," / ",len(fileNames)," Adding File ",fName)
    file=rt.TFile(fName)
    RateForTurnons  = file.Get('RateForTurnons')
    EtForTurnons    = file.Get('EtForTurnons')
    AcceptanceForTurnons = file.Get('AcceptanceForTurnons')
    TurnOnProgressionDir = file.Get('turn_on_progression')
    RateProgressionDir = file.Get('rate_progression')
    data=updateDataStore(data,RateForTurnons,AcceptanceForTurnons,TurnOnProgressionDir,RateProgressionDir)
    TFILE_STORE[fName]=file
#     if count>2:
#         break
print()
print("Loaded ",count," Files")
print("Loaded ",len(data)," options")

### Run2 Turnon Store

Loads the Rate vs pt histogram from the root file. This histogram will be used to obtain the Et threshold for selcting the baseline turnon for comparisons ( when doing Fixed rate studies) . 

Loads All the availabe TurnsOns from Run2. Please customize the selection accodingly , so as to load the loose or tight turnons . The file loaded here is the output of the [turn_on_production.C](https://github.com/ats2008/EGTagAndProbe/blob/CMSSW_12_X/EGTagAndProbe/test/fitter/turn_on_production.C)

In [ ]:
fileRate=rt.TFile(Run2RateFile)
Run2RateHist=fileRate.Get('DoubleEG_rate_LooseIso')

_fileRun2TurnOns=rt.TFile(Run2TurnOnFile)
fileRun2TurnOns=_fileRun2TurnOns.Get('0')
Run2TurnOnMap={}
Run2AcceptanceMap={}
PassOptions={}
FailOptions={}
for ky in  fileRun2TurnOns.GetListOfKeys():
    hName=ky.GetName()
    if not hName.startswith('hist_L1Et'):
        continue
    if 'loose' not in hName:
        continue
    if 'def' in hName:
        continue
    et=int(hName.split('_')[1].replace('L1Et',''))
    if 'Pass' in hName:
        PassOptions[et]=fileRun2TurnOns.Get(hName)
    elif 'Fail' in hName:
        FailOptions[et]=fileRun2TurnOns.Get(hName)
    else:
        continue

for ky in  fileRun2TurnOns.GetListOfKeys():
    hName=ky.GetName()
    if 'loose' not in hName:
        continue
    if 'def' in hName:
        continue
#     print(hName)
    if 'Efficiency' not in hName:
        continue
    if 'L1Et' not in hName:
        continue
    if 'Graph_from' in hName:
        continue
    if 'Barrel' in hName:
        continue
    if 'ECap' in hName:
        continue
    eT=int(hName.split('_')[1].replace('L1Et','').replace('Efficiency',''))
    Run2TurnOnMap[eT]=rt.TGraphAsymmErrors(fileRun2TurnOns.Get(hName))
    acceptance=-1.0
    if eT not in PassOptions or eT not in FailOptions:
        print("\t\t Problem !! eT not found in pass and fain maps !! eT = ",eT)
    else :
        acceptance=PassOptions[eT].Integral()*1.0/(PassOptions[eT].Integral() + FailOptions[eT].Integral())
    Run2AcceptanceMap[eT]=acceptance
    print('For ',eT,' Adding ',hName, "Acceptance = ",acceptance)


#### Load the Files and data

In [ ]:
fList=open(condorFileList)
txt=fList.readlines()
fileNames=[]
for l in txt:
    fileNames.append(l[:-1])
fList.close()

In [ ]:
data={}
TFILE_STORE={}
count=0
for fName in fileNames:
    count+=1
    print(count," / ",len(fileNames)," Adding File ",fName)
    file=rt.TFile(fName)
    RateForTurnons  = file.Get('RateDForTurnons')
    EtForTurnons    = file.Get('EtForTurnons')
    AcceptanceForTurnons = file.Get('AcceptanceForTurnons')
    TurnOnProgressionDir = file.Get('turn_on_progression')
    RateProgressionDir = file.Get('rate_progression_double')
    data=updateDataStore(data,RateForTurnons,AcceptanceForTurnons,TurnOnProgressionDir,RateProgressionDir)
    TFILE_STORE[fName]=file
#     if count>2:
#         break
print()
print("Loaded ",count," Files")
print("Loaded ",len(data)," options")

### Testing the root macro functions

In [ ]:
opt=101
opt2=101
cvs=rt.TCanvas("cvs")
cvs.cd()
data[opt]['FixedEtMetrics'][10.0]['turnOn'].Draw()
data[opt2]['FixedEtMetrics'][10.0]['turnOn'].Draw("same")
cvs.Draw()

#isGoodTurnOn

t1=data[opt]['FixedEtMetrics'][10.0]['turnOn']
t2=data[opt2]['FixedEtMetrics'][10.0]['turnOn']

y=rt.isGoodTurnON(t2,t1,10,19)
print(y)

### Integral

eT=10
print(data[opt]['FixedEtMetrics'][eT])
turnOn=data[opt]['FixedEtMetrics'][eT]['turnOn']

area=rt.getIntegral(turnOn,eT,eT-4.0,eT+4.0,"pref",rt.nullptr,False)
print("area = ",area)

## Get Baseline and Best Option for Fixed Rates

In [ ]:
def getBaselineForFixedRate(fRate,Run2RateHist,Run2TurnOnMap,Run2AcceptanceMap,effMin=0.43,effMax=0.82):
    nBins=Run2RateHist.GetNbinsX()
    eT=-1
    idx=-1
    rtFixHere=-1e6
    for i in range(nBins):
        rate=Run2RateHist.GetBinContent(i)
        if(rate > fRate ):
            continue
        else:
            if (fRate - rate ) < (fRate - rtFixHere):
                rtFixHere=rate
                idx=i    
    if idx>=0:
        eT=int(Run2RateHist.GetBinCenter(idx))+10
    if eT < 0:
        print(" No thresold found with 'rate < Fixed Rate ' (",fRate,"kHz in run 2 ")
        return (rt.nullptr,-1,-1,-1)
    if eT not in Run2TurnOnMap:
        print(" No Turnons found for thresold eT = ",eT," in run2 turn on collection")
        return (rt.nullptr,-1,-1,-1)
    
    turnon=Run2TurnOnMap[eT]
    nBins=turnon.GetN()
    idxA=-10
    idxB=-10
    for i in range(nBins):
        eff=turnon.GetPointY(i)
        if(eff > effMin and idxA < 0):
            idxA=i
        if(eff > effMax and idxB < 0):
            idxB=i
    if(idxB <0  or idxA <0 ):
        print("Something wrong with the idx finding for fRate ",fRate," which gave eT ",eT)
    print("Et for fixed rate ",fRate," -> ",int(Run2RateHist.GetBinCenter(idx)))
    print("\t Baseline found to be at eT = ",eT," for fixed Rate ",fRate,
              " indexes : ",idxA,"(","{0:0.3f}".format(turnon.GetPointY(idxA)),"/ ",effMin,")",
          idxB,"(","{0:0.3f}".format(turnon.GetPointY(idxB))," / ",effMax,")" )
    if eT in Run2AcceptanceMap:
        accep=Run2AcceptanceMap[eT]
    else:
        accep=-1.0
    return (Run2TurnOnMap[eT],idxA,idxB,eT,accep)
    
    
dataForValidation={'baseLine':[],'p1':[],'p2':[],'rslt':[],'turnOn':[]}

def getAllOptionsDataForFixedRate(data,F_Rate,Baseline=None):
    for key in dataForValidation:
        dataForValidation[key].clear()
    dataForFixedRate={'option':[],
                      'option_pars':[],
                      'eT':[],'eTLowEtLeg':[],
                      'rate':[],
                      'area':[],
                      'acceptance':[],
                      'turnOnLowEtLeg':[],
                      'turnOn':[],
                      'isGoodTurnOn':[],
                      'rateProgression':[]}
    for opt in data:
        idx=-1
        rateFixHere=-1e6
        eTmaxSearched=-1.0
        rateForETmaxSearched=-1.0
        for rate in data[opt]['FixedRateMetrics']:
            if eTmaxSearched < data[opt]['FixedRateMetrics'][rate]['eT']:
                eTmaxSearched=data[opt]['FixedRateMetrics'][rate]['eT']
                rateForETmaxSearched=rate
            if rate > F_Rate:
                continue
            else:
                if (F_Rate - rate ) < (F_Rate - rateFixHere):
                    rateFixHere=rate
        if(rateFixHere < 0.0 ):
            print("\tNo Et thresolds could be reached for option ",opt," at fixed Rate",F_Rate,
                  "( rate : ",rateForETmaxSearched," for eT ",eTmaxSearched," )" )
            continue
        vals=data[opt]['FixedRateMetrics'][rateFixHere]
        eT=vals['eT']+10
        isoLegTurnOn=data[opt]['FixedEtMetrics'][eT]['turnOn']
        isGoodTurnON=False #
        
        if Baseline[0]:
            isGoodTurnON=rt.isGoodTurnON(Baseline[0],isoLegTurnOn,Baseline[1],Baseline[2])
        
        dataForFixedRate['option'].append(opt)
        dataForFixedRate['eT'].append(eT)
        dataForFixedRate['eTLowEtLeg'].append(vals['eT'])
        dataForFixedRate['rate'].append(vals['_rate'])
        dataForFixedRate['area'].append(vals['area'])
        dataForFixedRate['isGoodTurnOn'].append(isGoodTurnON)
        dataForFixedRate['acceptance'].append(vals['acceptance'])
        dataForFixedRate['turnOn'].append(isoLegTurnOn)
        dataForFixedRate['turnOnLowEtLeg'].append(vals['turnOn'])
        dataForFixedRate['rateProgression'].append(data[opt]['rateProgression'])
        dataForFixedRate['option_pars'].append(data[opt]['params'])
        
    return dataForFixedRate

## Scanning the fixed rates

In [ ]:
fixedRates=[0.5,1,1.5,2,2.5,3,3.5,3.9,4.0,4,5,5,6]
fixedRates=[3,3.5,3.9,4.0,4,5]

In [ ]:
dataForFixedRates={}
effMin=0.85
effMax=0.95
OtherInfo+='Efficeiecy range in which isGoodTurnOn is ran : [ '+str(effMin)+','+str(effMax)+' ]\n'
for fR in fixedRates:
    print("Doing for Fixed Rate : ",fR)
    Baseline=getBaselineForFixedRate(fR,Run2RateHist,Run2TurnOnMap,Run2AcceptanceMap,effMin,effMax)
#     continue
    dataForFixedRates[fR]={}
    dataForFixedRates[fR]['Baseline']=Baseline
    dataForFixedRates[fR]['scan']=getAllOptionsDataForFixedRate(data,fR,Baseline)

## Export the results

In [ ]:
metricStr='area'
areaTol=0.8
accepTol=0.8
if not os.path.exists(prefixBase):
    print("making : ",prefixBase)
    os.system('mkdir -p '+prefixBase)
else:
    print("removing : ",prefixBase)
    os.system('rm -r '+prefixBase)
    print("making : ",prefixBase)
    os.system('mkdir -p '+prefixBase)
    
summaryFileName=prefixBase+'/'+'summaryForFixedRateScans.txt'
summaryFile=open(summaryFileName,'w')
summaryFile.write(DatsetInfoStr+'\n\n')
summaryFile.write(OtherInfo+'\n\n')

for fRate in dataForFixedRates:
    summaryFile.write('='*20+'\n')
    summaryFile.write("Summary at Fixed Rate = "+str(fRate)+" : \n \n")
    
    prefix=prefixBase+'/'+str(fRate)+'kHz'
    print("For Fixed rate ",fRate," Saving results to ",prefix)
    
    if not os.path.exists(prefix):
        print("making : ",prefix)
        os.system('mkdir -p '+prefix)
        
    #################   Baseline Details to the summary ###########
    
    summaryFile.write("  Baseline details the fixed rate \n")
    BaselineTurnOn=dataForFixedRates[fRate]['Baseline'][0]
    eTBaseLine=dataForFixedRates[fRate]['Baseline'][3]
    areaBaseline=rt.getIntegral(BaselineTurnOn,eTBaseLine,eTBaseLine-7.0,eTBaseLine+7.0,prefix+'/Baseline_',rt.nullptr,True)
    acceptanceBaseline=dataForFixedRates[fRate]['Baseline'][4]
    baselineString ='\tBaseline eT   : '+str(eTBaseLine) 
    baselineString+='\tBaseline Area : '+str(areaBaseline)
    baselineString+='\tBaseline Acceptance   : '+str(acceptanceBaseline)
    summaryFile.write(baselineString)
    summaryFile.write("\n\n")
    
    #####################################################################
    
    fixedRateData=dataForFixedRates[fRate]['scan']
    metric=np.asarray(fixedRateData[metricStr])
    sortedIdx=np.argsort(-1*metric)
    
    isSaturatingEarly=np.asarray(fixedRateData['isGoodTurnOn'])==True 
    hasBetterAreaMask=np.asarray(fixedRateData['area']) >= areaBaseline
    isFeasibleTurnONMask=(np.asarray(fixedRateData['eT']) - eTBaseLine) < 2.2
    
    isGoodTurnONMask = np.logical_and(isSaturatingEarly,isFeasibleTurnONMask)
    isGoodTurnONMask = hasBetterAreaMask
    isGoodTurnONMask = np.logical_and(hasBetterAreaMask,isFeasibleTurnONMask)
    isGoodTurnONMask = isSaturatingEarly
    print("\t\t isGoodTurnONMask  : ",sum(isGoodTurnONMask)," / ",len(isGoodTurnONMask))

    allOptResults=open(prefix+'/allOptionsScan.txt','w')
    allOptResults.write("Full options scans at fixed Rate "+str(fRate)+' kHz\n')
    allOptResults.write("In decending order of the metric "+metricStr+'\n')
    allOptResults.write(baselineString+'\n')
    allOptResults.write(OtherInfo+'\n')
    
    allOptResultsWithSelection=open(prefix+'/selectedOptions.txt','w')
    allOptResultsWithSelection.write("Full options scans at fixed Rate "+str(fRate)+' kHz\n')
    allOptResultsWithSelection.write("In decending order of the metric "+metricStr+'\n')
    allOptResultsWithSelection.write(baselineString+'\n')
    sectedOptionsFile = rt.TFile(prefix+'/selectedOptions.root','RECREATE') 
    baselineDir=sectedOptionsFile.mkdir("baselineDir")
    baselineDir.cd()
    BaselineTurnOn.Write()
    Run2RateHist.Write()
    
    allOptionsFile = rt.TFile(prefix+'/allOptions.root','RECREATE')
    baselineDir=allOptionsFile.mkdir("baselineDir")
    baselineDir.cd()
    BaselineTurnOn.Write()
    keysToPrint=['option','option_pars','eTLowEtLeg','eT','rate','isGoodTurnOn','area','acceptance']
    strToW=''
    
    for j in range(len(keysToPrint)):
        ky=keysToPrint[j]
        strToW+=ky
        strToW+="\t"
    strToW+='\n'
    allOptResults.write(strToW)
    allOptResultsWithSelection.write(strToW)
    
    eTsInTheScan=np.unique(np.asarray(fixedRateData['eT'])  ) 
    topOptionIdxForEachEt={i:[] for i in eTsInTheScan}
    count=0
    for i in sortedIdx:
        strToW=''
        for j in range(len(keysToPrint)):
            ky=keysToPrint[j]
            strToW+=str(fixedRateData[ky][i])
            strToW+="\t"
        strToW+='\n'
        allOptResults.write(strToW)
                
        opt=fixedRateData['option'][i]
        cDir=allOptionsFile.mkdir(str(opt))
        cDir.cd()
        fixedRateData['turnOn'][i].Write()
        fixedRateData['rateProgression'][i].Write()
        allTurnOns=cDir.mkdir("allTurnons")
        allTurnOns.cd()
#         for eT in data[opt]['FixedEtMetrics']:
#             if eT<22 or eT>28:
#                 continue
#             if data[opt]['FixedEtMetrics'][eT]['area']< 0.7*areaBaseline:
#                 continue
#             data[opt]['FixedEtMetrics'][eT]['turnOn'].Write()
        
        eT=fixedRateData['eT'][i]
        topOptionIdxForEachEt[eT].append(i)
        
#         if not isGoodTurnONMask[i]:
#             continue;
        if fixedRateData['eT'][i] > eTBaseLine+2:
            continue
        if fixedRateData['area'][i] < areaTol*areaBaseline:
            continue
        if fixedRateData['acceptance'][i] < acceptanceTol*acceptanceBaseline:
            continue
        count+=1
        strToW=''
        
        for j in range(len(keysToPrint)):
            ky=keysToPrint[j]
            strToW+=str(fixedRateData[ky][i])
            strToW+="\t"
        strToW+='\n'
        allOptResultsWithSelection.write(strToW)
    allOptResults.close()
    allOptResultsWithSelection.close()
    allOptionsFile.Close()
    print("  Number of options passing the section " ,count)
    summaryFile.write("\n  Number of options passing the selection : "+str(count)+"\n\n")
    
    
    #################   TOP n Options   to Summary ################
    summaryFile.write("  Best options for the fixed rate \n")
    count=0
    strToW='\t'
    top5opt_dir=sectedOptionsFile.mkdir("top_five_opions")
    top5opt_dir.cd()
    for j in range(len(keysToPrint)):
        strToW+=keysToPrint[j]+'\t'
    summaryFile.write(strToW+'\n')
    
    nMax=5
    for i in sortedIdx:
#         if not isGoodTurnONMask[i]:
#             continue;
        if fixedRateData['eT'][i] > eTBaseLine+2:
    #             print(fixedRateData['eT'][i] ," > ", eTBaseLine+2)
            continue
        if fixedRateData['area'][i] < areaTol*areaBaseline:
#             print( "area :",fixedRateData['area'][i] ,"  < ",1.0*areaBaseline)
            continue
        if fixedRateData['acceptance'][i] < acceptanceTol*acceptanceBaseline:
#             print( "area :",fixedRateData['acceptance'][i]  ,"  < ",acceptanceTol*acceptanceBaseline)
            continue
            
        count+=1
        area=rt.getIntegral(fixedRateData['turnOn'][i],eT,eT-7.0,eT+7.0,prefix+'/top'+str(nMax)+'_',BaselineTurnOn,True)
        if count > nMax :
            break
        strToW='\t'
        for j in range(len(keysToPrint)):
            strToW+=str(fixedRateData[keysToPrint[j]][i])+"\t"
        summaryFile.write(strToW+'\n')
                
        cDir=top5opt_dir.mkdir(str(count))
        cDir.cd()
        fixedRateData['turnOn'][i].Write()
        fixedRateData['rateProgression'][i].Write()
         
        allTurnOns=cDir.mkdir("allTurnons")
        allTurnOns.cd()
        opt=fixedRateData['option'][i]
        for eT in data[opt]['FixedEtMetrics']:
            data[opt]['FixedEtMetrics'][eT]['turnOn'].Write()
            
    summaryFile.write("\n")
    
    #################   TOP n Options for unique eT  to Summary ################
    nMax=3
    top3OptInPt_dir=sectedOptionsFile.mkdir("top_three_options_in_Et")
    for eT in eTsInTheScan:
        summaryFile.write("  Best options for eT : [ "+str(eT-10)+","+str(eT)+' ]\n')
        count=0
        strToW='\t'
        for j in range(len(keysToPrint)):
            strToW+=keysToPrint[j]+'\t'
        summaryFile.write(strToW+'\n')
        forCurrentPt_dir=top3OptInPt_dir.mkdir("eT"+str(eT))
        forCurrentPt_dir.cd()

        for i in topOptionIdxForEachEt[eT]:
            
#             if not isGoodTurnONMask[i]:
#                 continue;
            count+=1
            area=rt.getIntegral(fixedRateData['turnOn'][i],eT,eT-7.0,eT+7.0,prefix+'/top'+str(nMax)+'inPtBin_',
                                BaselineTurnOn,True)
            if count > nMax :
                break
            strToW='\t'
            for j in range(len(keysToPrint)):
                strToW+=str(fixedRateData[keysToPrint[j]][i])+"\t"
            summaryFile.write(strToW+'\n')
            
            cDir=forCurrentPt_dir.mkdir(str(count))
            cDir.cd()
            fixedRateData['turnOn'][i].Write()
            fixedRateData['rateProgression'][i].Write()

            allTurnOns=cDir.mkdir("allTurnons")
            allTurnOns.cd()
            opt=fixedRateData['option'][i]
            for e in data[opt]['FixedEtMetrics']:
                data[opt]['FixedEtMetrics'][e]['turnOn'].Write()
    sectedOptionsFile.Close()
        
summaryFile.close() 
print('\nsummary written into ',summaryFileName)